In [1]:
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

import random
import numpy as np
import matplotlib.pyplot as plt

# random seedを設定
seed = 2023
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [2]:
# 画像をテンソルに変換
data_transform = transforms.Compose([
    transforms.ToTensor()
])

In [3]:
from tensorflow.keras.datasets import cifar10

# CIFAR-10データセットのダウンロード
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# データの確認
print(f"訓練データ: {x_train.shape}, 訓練ラベル: {y_train.shape}")
print(f"テストデータ: {x_test.shape}, テストラベル: {y_test.shape}")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 383s 2us/step
訓練データ: (50000, 32, 32, 3), 訓練ラベル: (50000, 1)
テストデータ: (10000, 32, 32, 3), テストラベル: (10000, 1)


In [ ]:
train_size = int(0.8 * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_val_dataset, [train_size, val_size])


In [7]:
from torch.utils.data import DataLoader
BATCH_SIZE = 64  # ミニバッチサイズを設定、データセットのサイズによって自分で設定する
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

NameError: name 'train_dataset' is not defined

In [ ]:
# ハードウェアの設定 
device = torch.device('mps')

In [ ]:
# ニューラルネットワークに入力する画像のサイズ、チャネル数をチェック
images, labels = next(iter(train_loader))
c, h, w = images[0].shape
print("ミニバッチサイズ: ", images.size())
print("画像のチャネル数: ", c)
print("画像の高さ: ", h)
print("画像の幅: ", w)

# グリッド乗に4枚の画像を表示
img = torchvision.utils.make_grid(images[:4])
img = transforms.functional.to_pil_image(img)
display(img)

# 上の画像に対応するラベルを表示
print(labels[:4])

In [ ]:
# ニューラルネットワークに入力する画像のサイズと分類するクラス数を定義
INPUT_CHANNELS = c
OUTPUT_SIZE = 10  # 分類するクラス数

# RestNet152をロード
model = torchvision.models.mobilenet_v2(weights="IMAGENET1K_V1")

# 最終層の出力ユニット数を変更
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, OUTPUT_SIZE)

# モデルをデバイスに転送
model = model.to(device)
model

In [ ]:
# 多クラス分類のための損失関数を定義
criterion = nn.CrossEntropyLoss()

In [ ]:
# モデルのパラメータを更新するための最適化手法をAdamで定義
optimizer = optim.Adam(model.parameters(), lr=0.001)